In [1]:
# To install matlab, run pip install matlab
# To install matlab engine, run pip install 'matlabengine == 9.13.7'

import numpy as np
from scipy.io import loadmat
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA
from Models import BG


In [2]:
mat = loadmat('tsd180.mat')
d = mat['days']
p = mat['pm']
n = mat['nmss']
[N,M] = np.shape(p)

Data = pd.DataFrame({'days': d[:,0]})
for i in range(M):
    Datai = pd.DataFrame({n[i,0][0] : p[:,i]})
    Data = pd.concat([Data,Datai],axis=1)

DataETFs = Data[['spy', 'xle', 'xlf', 'xli', 'xlk', 'xlp', 'xlu', 'xlv', 'xly', 'xom', 'xrx']]
DataETFsReturns = DataETFs.diff()
DataETFsReturns = DataETFsReturns.drop(index = 0)
DataETFsReturns.insert(0, 'days', d[1:])
DataETFs.insert(0, 'days', d)
DataETFsReturns['days'] = pd.to_datetime(DataETFsReturns['days'], format='%Y%m%d')

X = DataETFsReturns.iloc[:,1:].values

#DataETFs.head(10)
print(X.shape)
print(X)

(4430, 11)
[[ 0.63    0.0485  0.1218 ...  0.08   -0.59    0.7377]
 [-0.14   -0.6117 -0.0244 ...  0.19   -0.56   -0.1317]
 [ 0.48   -0.8739  0.1218 ...  0.21   -1.1     0.    ]
 ...
 [ 0.86   -0.38    0.12   ...  0.62   -0.48    0.13  ]
 [ 0.18   -1.3     0.11   ... -0.94   -1.47   -0.36  ]
 [-1.38   -0.19   -0.12   ... -1.11   -0.21   -0.25  ]]


In [3]:
import os
print(os.cpu_count())

22


In [4]:
# 1) Build one BG instance with FFT grid on [−10%, +10%]:
bg = BG(N=4096, Xmax=0.1)

# 2) Fit SPY (column 0) *serially* (no per-window parallel):
# spy_params = bg.fit_series(X[:, 0], window=100)
# print("SPY params shape:", spy_params.shape)
# print("Example SPY at t=150:", spy_params[150, :])

# 3) Fit all 11 assets *in parallel across assets* (1 process per asset):
all_params = bg.fit_multiple(X, window=100, n_workers=11)
print("All params shape:", all_params.shape)
print("Example (t=150, asset=3):", all_params[150, 3, :])

All params shape: (4430, 11, 4)
Example (t=150, asset=3): [2.09822984e-01 1.48767576e-01 1.40261570e-05 8.48533783e+01]


In [5]:
import os
import numpy as np

# Create the folder if it doesn't exist
os.makedirs('estimates', exist_ok=True)

# Save the array
np.save('estimates/all_params.npy', all_params)

# To load it later:
# loaded_params = np.load('estimates/all_params.npy')

In [6]:
loaded_params = np.load('estimates/all_params.npy')
print("Example (t=150, asset=3):", loaded_params[150, 3, :])

Example (t=150, asset=3): [2.09822984e-01 1.48767576e-01 1.40261570e-05 8.48533783e+01]
